In [29]:
import numpy as np
import pandas as pd
import datetime
import re

In [70]:
raw = pd.read_csv('sample.csv', encoding='UTF-8',
                #dtype={'FooDuration', pd.Timedelta}
                 )

In [71]:
raw

,Timestamp,Address,ZIP,FullName,FooDuration,BarDuration,TotalDuration,Notes
0,4/1/11 11:00:00 AM,"123 4th St, Anywhere, AA",94121,Monkey Alberto,1:23:32.123,1:32:33.123,zzsasdfa,I am the very model of a modern major general
1,3/12/14 12:00:00 AM,"Somewhere Else, In Another Time, BB",1,Superman übertan,111:23:32.123,1:32:33.123,zzsasdfa,This is some Unicode right here. ü ¡! 😀
2,2/29/16 12:11:11 PM,111 Ste. #123123123,1101,Résumé Ron,31:23:32.123,1:32:33.123,zzsasdfa,🏳️🏴🏳️🏴
3,1/1/11 12:00:01 AM,"This Is Not An Address, BusyTown, BT",94121,Mary 1,1:23:32.123,0:00:00.000,zzsasdfa,I like Emoji! 🍏🍎😍
4,12/31/16 11:59:59 PM,"123 Gangnam Style Lives Here, Gangnam Town",31403,Anticipation of Unicode Failure,1:23:32.123,1:32:33.123,zzsasdfa,I like Math Symbols! ≱≰⨌⊚
5,11/11/11 11:11:11 AM,überTown,10001,Prompt Negotiator,1:23:32.123,1:32:33.123,zzsasdfa,"I’m just gonna say, this is AMAZING. WHAT NEGO..."
6,5/12/10 4:48:12 PM,Høøük¡,1231,Sleeper Service,1:23:32.123,1:32:33.123,zzsasdfa,2/1/22
7,10/5/12 10:31:11 PM,"Test Pattern Town, Test Pattern, TP",121,株式会社スタジオジブリ,1:23:32.123,1:32:33.123,zzsasdfa,1:11:11.123
8,10/2/04 8:44:11 AM,The Moon,11,HERE WE GO,1:23:32.123,1:32:33.123,zzsasdfa,NaN


In [84]:
def convert_interval(intv_str):
    """Assume format is <HHH:MIN:SEC.MSEC>. 
    It's easier to use a regex to parse this which allows a colon or period to separate fields.
    If data was not well formed this could cause errors.
    """
    # check to see if we've already converted it e.g. we're re-running this
    if isinstance(intv_str, datetime.timedelta):
        return intv_str
    _i = re.compile('[:.]').split(intv_str)
    _i = list(map(lambda x: int(x), _i))
    return datetime.timedelta(hours=_i[0], minutes=_i[1], seconds=_i[2], milliseconds=_i[3])

In [82]:
# Convert string interval into a Timedelta object
raw['FooDuration'] = raw.FooDuration.apply(convert_interval)
raw['BarDuration'] = raw.BarDuration.apply(convert_interval)

In [83]:
raw['TotalDuration'] = raw.BarDuration + raw.FooDuration

In [78]:
# wrap x in an int so we can re-run this without errors (idempotent)
raw.ZIP = raw.ZIP.apply(lambda x: f"{int(x):05d}")

In [85]:
raw

,Timestamp,Address,ZIP,FullName,FooDuration,BarDuration,TotalDuration,Notes
0,4/1/11 11:00:00 AM,"123 4th St, Anywhere, AA",94121,Monkey Alberto,0 days 01:23:32.123000,01:32:33.123000,0 days 02:56:05.246000,I am the very model of a modern major general
1,3/12/14 12:00:00 AM,"Somewhere Else, In Another Time, BB",00001,Superman übertan,4 days 15:23:32.123000,01:32:33.123000,4 days 16:56:05.246000,This is some Unicode right here. ü ¡! 😀
2,2/29/16 12:11:11 PM,111 Ste. #123123123,01101,Résumé Ron,1 days 07:23:32.123000,01:32:33.123000,1 days 08:56:05.246000,🏳️🏴🏳️🏴
3,1/1/11 12:00:01 AM,"This Is Not An Address, BusyTown, BT",94121,Mary 1,0 days 01:23:32.123000,00:00:00,0 days 01:23:32.123000,I like Emoji! 🍏🍎😍
4,12/31/16 11:59:59 PM,"123 Gangnam Style Lives Here, Gangnam Town",31403,Anticipation of Unicode Failure,0 days 01:23:32.123000,01:32:33.123000,0 days 02:56:05.246000,I like Math Symbols! ≱≰⨌⊚
5,11/11/11 11:11:11 AM,überTown,10001,Prompt Negotiator,0 days 01:23:32.123000,01:32:33.123000,0 days 02:56:05.246000,"I’m just gonna say, this is AMAZING. WHAT NEGO..."
6,5/12/10 4:48:12 PM,Høøük¡,01231,Sleeper Service,0 days 01:23:32.123000,01:32:33.123000,0 days 02:56:05.246000,2/1/22
7,10/5/12 10:31:11 PM,"Test Pattern Town, Test Pattern, TP",00121,株式会社スタジオジブリ,0 days 01:23:32.123000,01:32:33.123000,0 days 02:56:05.246000,1:11:11.123
8,10/2/04 8:44:11 AM,The Moon,00011,HERE WE GO,0 days 01:23:32.123000,01:32:33.123000,0 days 02:56:05.246000,NaN


In [7]:
# If run, this throws an error: "UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 28: invalid start byte"
# raw_bad = pd.read_csv('sample-with-broken-utf8.csv', encoding='UTF-8')

In [13]:
raw.dtypes

Timestamp        object
Address          object
ZIP               int64
FullName         object
FooDuration      object
BarDuration      object
TotalDuration    object
Notes            object
dtype: object